In [3]:
!pip3 install torchvision
!pip3 install torch_snippets
!pip3 install PIL

ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


In [4]:
import torch
from PIL import Image
from torchvision.models.detection import fasterrcnn_resnet50_fpn 
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights
import exp_utils
import matplotlib.pyplot as plt
import matplotlib
import glob

In [5]:
device = 'cpu'

In [6]:
# Loading images into PIL files

PIL_images = []
for filename in glob.glob("test_images/*"): 
	img = Image.open(filename)
	PIL_images.append(img)

In [7]:
# Loading Faster RCNN model

weights_rcnn = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
model_rcnn = fasterrcnn_resnet50_fpn(weights=weights_rcnn)
model_rcnn.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [8]:
# Preprocessing images for Faster RCNN

transforms_rcnn = weights_rcnn.transforms()
input_list_rcnn = []
for img in PIL_images:
	transformed_img = transforms_rcnn(img)
	input_list_rcnn.append(transformed_img)

In [9]:
print(input_list_rcnn)

[tensor([[[0.9765, 0.9804, 0.9804,  ..., 0.2000, 0.1843, 0.1882],
         [0.9804, 0.9804, 0.9804,  ..., 0.2353, 0.2118, 0.2000],
         [0.9804, 0.9804, 0.9804,  ..., 0.2471, 0.2392, 0.2314],
         ...,
         [0.8000, 0.8039, 0.8000,  ..., 0.2353, 0.2627, 0.2784],
         [0.8078, 0.8000, 0.7961,  ..., 0.2667, 0.2941, 0.3098],
         [0.8078, 0.7961, 0.8000,  ..., 0.2745, 0.2549, 0.3255]],

        [[0.9843, 0.9882, 0.9882,  ..., 0.1647, 0.1490, 0.1529],
         [0.9882, 0.9882, 0.9882,  ..., 0.2000, 0.1765, 0.1647],
         [0.9882, 0.9882, 0.9882,  ..., 0.2118, 0.2039, 0.1961],
         ...,
         [0.7569, 0.7608, 0.7569,  ..., 0.2275, 0.2549, 0.2706],
         [0.7647, 0.7569, 0.7529,  ..., 0.2588, 0.2863, 0.3020],
         [0.7647, 0.7529, 0.7569,  ..., 0.2667, 0.2471, 0.3176]],

        [[0.9725, 0.9765, 0.9765,  ..., 0.0980, 0.0824, 0.0863],
         [0.9765, 0.9765, 0.9765,  ..., 0.1333, 0.1098, 0.0980],
         [0.9765, 0.9765, 0.9765,  ..., 0.1451, 0.1373, 0

In [10]:
# Running images through Faster RCNN model

model_rcnn.eval()
results_rcnn = model_rcnn(input_list_rcnn)

In [11]:
print(results_rcnn)
print(type(results_rcnn))

[{'boxes': tensor([[2257.4214, 1587.8955, 4013.8032, 2778.9805],
        [ 866.9505,  148.8172, 3257.3628, 2363.6753],
        [2800.8789,  221.7952, 3261.2092,  371.0679],
        [2731.4988,  268.5606, 3170.6653,  403.3899],
        [2707.8672,  307.1399, 3119.0449,  468.6767],
        [   0.0000,    0.0000, 3932.1201, 2993.4221],
        [ 932.1949,   65.1756, 1624.6669,  655.1740],
        [ 765.2560,  622.9561,  891.5738,  798.1429],
        [ 622.9224,  737.0305,  878.7100, 1036.9211],
        [ 807.0602, 2372.8721,  991.7570, 2557.8943],
        [ 974.4955,  530.5814, 3995.8970, 2944.8391],
        [3545.7615,   21.5120, 3999.3862,  259.9743],
        [ 993.7166, 2374.8755, 1170.6838, 2543.1067],
        [   0.0000, 1915.9077, 3548.2422, 2964.9812],
        [ 646.2650, 2191.9917,  826.7542, 2373.9871],
        [ 240.7716, 2547.7429, 3227.9854, 3012.9158],
        [ 766.9217,  628.4650, 1022.4523,  819.3538],
        [ 457.2944, 2392.6609,  589.3725, 2583.2241],
        [ 454.662

In [12]:
cmap = matplotlib.cm.get_cmap('tab20')
score_threshold = 0.5

/tmp/ipykernel_6187/4270097320.py:1: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = matplotlib.cm.get_cmap('tab20')


In [13]:
# Visualizing Faster RCNN results

classes_rcnn = exp_utils.get_fasterrnn_classes()

for i in range(len(results_rcnn)):
	result = results_rcnn[i]
	scores = result['scores']
	boxes = result['boxes'][scores > score_threshold]
	labels = result['labels'][scores > score_threshold]
	scores = result['scores'][scores > score_threshold]
	curr_img = PIL_images[i]
	exp_utils.show(curr_img, boxes, scores, labels, cmap, classes_rcnn)
	path = f"qualitative_outputs/{i}_rcnn.jpg"
	plt.savefig(path)

TypeError: can only be called with ndarray object

Error in callback <function _draw_all_if_interactive at 0x15ebca980> (for post_execute):


ValueError: object __array__ method not producing an array

ValueError: output array must be a NumPy array

<Figure size 640x480 with 1 Axes>

In [ ]:
# Loading YOLOv5 model

model_yolo = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model_yolo.to(device)

In [ ]:
# Running images through YOLOv5 model

results_yolo = []

for img in PIL_images:
	curr_result = model_yolo(img)
	results_yolo.append(curr_result)

In [ ]:
print(results_yolo)
print(type(results_yolo))

In [ ]:
# Visualizing YOLOv5 results

classes_yolo = model_yolo.names

for i in range(len(results_yolo)):
	result = results_yolo[i]
	scores = result.xyxy[0][:,4]
	boxes = result.xyxy[0][:,:4][scores > score_threshold]
	labels = result.xyxy[0][:,5][scores > score_threshold]
	scores = result.xyxy[0][:,4][scores > score_threshold]
	curr_img = PIL_images[i]
	exp_utils.show(curr_img, boxes, scores, labels, cmap, classes_yolo)
	path = f"qualitative_outputs/{i}_yolo.jpg"
	plt.savefig(path)